# Valle del Cauca Exports - Transformation

## Dependencies

In [1]:
import os
import sys
import duckdb

sys.path.append("../ingest")
import config  # type: ignore

## Duckdb Database

In [ ]:
if not os.path.exists(os.path.join(config.Database.dir, config.Database.filename)):
    os.makedirs(config.Database.dir, exist_ok=True)

ddb = duckdb.connect(os.path.join(config.Database.dir, config.Database.filename))

In [3]:
ddb.execute(
    """CREATE OR REPLACE TABLE valle_exports
    AS SELECT * FROM '../data/exports/clean/clean_exports.parquet';
    """
)

## Dataset Summary

In [4]:
ddb.sql("SUMMARIZE valle_exports")

┌─────────────┬─────────────┬─────────┬──────────────────┬───────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬───────────────────┬────────┬─────────────────┐
│ column_name │ column_type │   min   │       max        │ approx_unique │        avg         │        std         │        q25         │        q50         │        q75        │ count  │ null_percentage │
│   varchar   │   varchar   │ varchar │     varchar      │     int64     │      varchar       │      varchar       │      varchar       │      varchar       │      varchar      │ int64  │  decimal(9,2)   │
├─────────────┼─────────────┼─────────┼──────────────────┼───────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼───────────────────┼────────┼─────────────────┤
│ COD_PAI4    │ VARCHAR     │ ABW     │ ZWE              │           206 │ NULL               │ NULL               │ NULL               │ NULL               │ NULL             

## Top Departments by Exports Value (COP)

- Antioquia = 5.
- Cesar = 20.
- Bogotá = 11.
- Guajira = 44.
- Cundinamarca = 25.
- Valle del Cauca = 76.
- Bolivar = 13.
- Atlantico = 8.
- Santander = 68.
- Casanare = 85.

In [5]:
ddb.execute(
    """CREATE OR REPLACE TABLE top_valle_dptos
    AS SELECT DPTO1, SUM(FOBPES) TOTAL_FOBPES
    FROM valle_exports
    WHERE MODAD = 198
    GROUP BY DPTO1
    HAVING DPTO1 <> 0
    ORDER BY TOTAL_FOBPES desc;"""
)

In [6]:
ddb.sql("SELECT * FROM top_valle_dptos LIMIT 10")

┌───────┬───────────────────┐
│ DPTO1 │   TOTAL_FOBPES    │
│ uint8 │      double       │
├───────┼───────────────────┤
│     5 │ 274263589989446.0 │
│    20 │ 162419056332081.0 │
│    11 │ 137117742546987.0 │
│    44 │  97596420351992.0 │
│    25 │  88827005479502.0 │
│    76 │  81858334750492.0 │
│    13 │  70896218259780.0 │
│     8 │  58626382716991.0 │
│    68 │  45415433606442.0 │
│    85 │  42147727224518.0 │
├───────┴───────────────────┤
│ 10 rows         2 columns │
└───────────────────────────┘

## Top Valle del Cauca Exported Goods by Value (COP)
- SUGARS AND SUGAR CONFECTIONERY = **17**.
- ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS THEREOF; SOUND RECORDERS AND REPRODUCERS; TELEVISION IMAGE AND SOUND RECORDERS AND REPRODUCERS, PARTS AND ACCESSORIES OF SUCH ARTICLES = **85**.
- ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSMETIC OR TOILET PREPARATIONS = **33**.
- COFFEE, TEA, MATE AND SPICES = **09**.
- ANIMAL OR VEGETABLE FATS AND OILS AND THEIR CLEAVAGE PRODUCTS; PREPARED ANIMAL FATS; ANIMAL OR VEGETABLE WAXES = **15**.
- PAPER AND PAPERBOARD; ARTICLES OF PAPER PULP, OF PAPER OR PAPERBOARD = **48**.
- PHARMACEUTICAL PRODUCTS = **30**.
- PLASTICS AND ARTICLES THEREOF = **39**.
- ORGANIC CHEMICALS = **29**.
- SOAP, ORGANIC SURFACE-ACTIVE AGENTS; WASHING, LUBRICATING, POLISHING OR SCOURING PREPARATIONS; ARTIFICIAL OR PREPARED WAXES, CANDLES AND SIMILAR ARTICLES, MODELLING PASTES, DENTAL WAXES AND DENTAL PREPARATIONS WITH A BASIS OF PLASTER = **34**.

In [7]:
ddb.execute(
    """CREATE OR REPLACE TABLE top_valle_exports
    AS SELECT POSAR, SUM(FOBPES) TOTAL_FOBPES
    FROM valle_exports
    WHERE MODAD = 198
    AND DPTO1 = 76
    GROUP BY POSAR
    ORDER BY TOTAL_FOBPES desc;"""
)

In [8]:
ddb.sql("SELECT * FROM top_valle_exports LIMIT 10")

┌───────┬──────────────────┐
│ POSAR │   TOTAL_FOBPES   │
│ uint8 │      double      │
├───────┼──────────────────┤
│    17 │ 12702262350103.0 │
│    85 │  8485086383440.0 │
│    33 │  6103203042884.0 │
│     9 │  5539128768931.0 │
│    15 │  5434360512784.0 │
│    48 │  4743965456524.0 │
│    30 │  4719139376023.0 │
│    39 │  2842333944553.0 │
│    29 │  2773144872237.0 │
│    34 │  2720894673315.0 │
├───────┴──────────────────┤
│ 10 rows        2 columns │
└──────────────────────────┘

## Top Valle del Cauca Exported Goods by Agregated Value (COP)

- SUGARS AND SUGAR CONFECTIONERY = **17**.
- ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSMETIC OR TOILET PREPARATIONS = **33**.
- ORGANIC CHEMICALS = **29**.
- ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS THEREOF; SOUND RECORDERS AND REPRODUCERS; TELEVISION IMAGE AND SOUND RECORDERS AND REPRODUCERS, PARTS AND ACCESSORIES OF SUCH ARTICLES = **85**.
- RUBBER AND ARTICLES THEREOF = **40**.
- PHARMACEUTICAL PRODUCTS = **30**.
- APPAREL AND CLOTHING ACCESSORIES; KNITTED OR CROCHETED = **61**.
- PLASTICS AND ARTICLES THEREOF = **39**.
- FURNITURE; BEDDING, MATTRESSES, MATTRESS SUPPORTS, CUSHIONS AND SIMILAR STUFFED FURNISHINGS; LAMPS AND LIGHTING FITTINGS, N.E.C.; ILLUMINATED SIGNS, ILLUMINATED NAME-PLATES AND THE LIKE; PREFABRICATED BUILDINGS = **94**.
- APPAREL AND CLOTHING ACCESSORIES; NOT KNITTED OR CROCHETED = **62**.


In [9]:
ddb.execute(
    """CREATE OR REPLACE TABLE top_valle_agrena
    AS SELECT POSAR, SUM(AGRENA) TOTAL_AGRENA
    FROM valle_exports
    WHERE MODAD = 198
    AND DPTO1 = 76
    GROUP BY POSAR
    ORDER BY TOTAL_AGRENA desc;"""
)

In [10]:
ddb.sql("SELECT * FROM top_valle_agrena LIMIT 10")

┌───────┬──────────────┐
│ POSAR │ TOTAL_AGRENA │
│ uint8 │    double    │
├───────┼──────────────┤
│    17 │ 1393716772.0 │
│    33 │  686211359.0 │
│    29 │  530480555.0 │
│    85 │  522645456.0 │
│    40 │  415211786.0 │
│    30 │  365447795.0 │
│    61 │  238455051.0 │
│    39 │  208925305.0 │
│    94 │   87378358.0 │
│    62 │   86717932.0 │
├───────┴──────────────┤
│ 10 rows    2 columns │
└──────────────────────┘

## Top Valle del Cauca Exports Destination by Value (COP)

- United States = USA.
- Ecuador = ECU.
- Peru = PER.
- Chile = CHL.
- Mexico = MEX.
- China = CHN.
- Venezuela = VEN.
- Dominican Republic = DOM.
- Panama = PAN.
- Brazil = BRA.

In [11]:
ddb.execute(
    """CREATE OR REPLACE TABLE top_valle_destinations
    AS SELECT COD_PAI4, SUM(FOBPES) TOTAL_FOBPES
    FROM valle_exports
    WHERE MODAD = 198
    AND DPTO1 = 76
    GROUP BY COD_PAI4
    ORDER BY TOTAL_FOBPES desc;"""
)

In [12]:
ddb.sql("SELECT * FROM top_valle_destinations LIMIT 10")

┌──────────┬──────────────────┐
│ COD_PAI4 │   TOTAL_FOBPES   │
│ varchar  │      double      │
├──────────┼──────────────────┤
│ USA      │ 19158754973403.0 │
│ ECU      │ 11946248278060.0 │
│ PER      │  6793737554809.0 │
│ CHL      │  4714338667662.0 │
│ MEX      │  4480139173605.0 │
│ CHN      │  4069893807759.0 │
│ VEN      │  2434020564921.0 │
│ DOM      │  2384178293877.0 │
│ PAN      │  2275455188170.0 │
│ BRA      │  2262575454994.0 │
├──────────┴──────────────────┤
│ 10 rows           2 columns │
└─────────────────────────────┘

## Top Valle del Cauca Exported Goods to Korea by Value (COP)

- COPPER AND ARTICLES THEREOF = **74**.
- PAPER AND PAPERBOARD; ARTICLES OF PAPER PULP, OF PAPER OR PAPERBOARD = **48**.
- COFFEE, TEA, MATE AND SPICES = **09**.
- SUGARS AND SUGAR CONFECTIONERY = **17**.
- PREPARATIONS OF VEGETABLES, FRUIT, NUTS OR OTHER PARTS OF PLANTS = **20**.
- ALUMINIUM AND ARTICLES THEREOF = **76**.
- MISCELLANEOUS EDIBLE PREPARATIONS = **21**.
- APPAREL AND CLOTHING ACCESSORIES; KNITTED OR CROCHETED = **61**.
- ANIMAL ORIGINATED PRODUCTS; NOT ELSEWHERE SPECIFIED OR INCLUDED = **05**.
- PRINTED BOOKS, NEWSPAPERS, PICTURES AND OTHER PRODUCTS OF THE PRINTING INDUSTRY; MANUSCRIPTS, TYPESCRIPTS AND PLANS = **49**.

In [13]:
ddb.execute(
    """CREATE OR REPLACE TABLE top_valle_exports_to_korea
    AS SELECT POSAR, SUM(FOBPES) TOTAL_FOBPES
    FROM valle_exports
    WHERE MODAD = 198
    AND DPTO1 = 76
    AND COD_PAI4 = 'KOR'
    GROUP BY POSAR
    ORDER BY TOTAL_FOBPES desc;"""
)

In [14]:
ddb.sql("SELECT * FROM top_valle_exports_to_korea LIMIT 10")

┌───────┬────────────────┐
│ POSAR │  TOTAL_FOBPES  │
│ uint8 │     double     │
├───────┼────────────────┤
│    74 │ 221841854647.0 │
│    48 │ 128270074264.0 │
│     9 │ 101644144120.0 │
│    17 │  34464214622.0 │
│    20 │  26086622810.0 │
│    76 │  12724974271.0 │
│    21 │   2244753550.0 │
│    61 │   1307548405.0 │
│     5 │    684778670.0 │
│    49 │    646734013.0 │
├───────┴────────────────┤
│ 10 rows      2 columns │
└────────────────────────┘

## Check all Tables

In [15]:
ddb.sql("SHOW ALL TABLES")

┌──────────┬─────────┬────────────────────────────┬─────────────────────────────────────────────────┬───────────────────────────────────────────────────────┬───────────┐
│ database │ schema  │            name            │                  column_names                   │                     column_types                      │ temporary │
│ varchar  │ varchar │          varchar           │                    varchar[]                    │                       varchar[]                       │  boolean  │
├──────────┼─────────┼────────────────────────────┼─────────────────────────────────────────────────┼───────────────────────────────────────────────────────┼───────────┤
│ db       │ main    │ top_valle_agrena           │ [POSAR, TOTAL_AGRENA]                           │ [UTINYINT, DOUBLE]                                    │ false     │
│ db       │ main    │ top_valle_destinations     │ [COD_PAI4, TOTAL_FOBPES]                        │ [VARCHAR, DOUBLE]                               

## Close Connection

In [16]:
ddb.close()